In [1]:
import tensorflow as tf

# get the GPU device
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
else:
  print('No GPU found')

Found GPU at: /device:GPU:0


In [2]:
import torch

if torch.cuda.is_available():

  # tell PyTorch to use the GPU
  device = torch.device('cuda')

  print("There are %d GPUs available" %torch.cuda.device_count())
  print('Use GPU:', torch.cuda.get_device_name(0))

There are 1 GPUs available
Use GPU: Tesla K80


In [3]:
! pip install Transformers

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 12.3MB/s 
     |████████████████████████████████| 890kB 20.1MB/s 
     |████████████████████████████████| 3.8MB 24.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c21e6ea30374d62357dc2b966ce5a39b3ba60ee2bc1bc49239d917f08da5565a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Loading the `Cola Dataset`

In [4]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=94eff99bd7dcdb646fec93ba439a266a4d0ca08ddfc60829ff8791021683f2d1
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [5]:
import os
import wget

url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

if not os.path.exists('./cola_public_1.1.zip'):
  wget.download(url, './cola_public_1.1.zip')
  #!wget url

for dirname, _, filenames in os.walk('./'):
  for filename in filenames:
    if 'cola' in filename:
      print(os.path.join(dirname, filename))

./cola_public_1.1.zip


In [6]:
# unzip the data
if not os.path.exists('./cola_public_1.1/'):
  !unzip ./cola_public_1.1.zip

Archive:  ./cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


In [7]:
import pandas as pd
df = pd.read_csv('./cola_public/tokenized/in_domain_train.tsv', sep='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

print(df.shape)
df.sample(10)

(8551, 4)


,sentence_source,label,label_notes,sentence
1556,r-67,0,*,"tom ordered bacon , and dick lettuce , and i t..."
8549,ad03,1,NaN,i had the strangest feeling that i knew you .
7569,sks13,1,NaN,bill 's mother saw him .
770,bc01,0,*,we love they .
7840,ad03,1,NaN,the therapist 's analysis of lucy
921,bc01,0,*,while holly did n't discuss a report about eve...
2501,l-93,1,NaN,pauline hammered the metal flat .
8250,ad03,0,*,gilgamesh might have been not reading the cune...
7510,sks13,0,*,john hurt john with john 's umbrella when john...
4801,ks08,1,NaN,john asks which book by his father he read .


We care about `sentence` and `label`
`label` = 0 , means not grammatically not acceptable
`label` = 1, means grammatrically acceptable

In [8]:
df.loc[df.label == 0].sample(5)[['sentence', 'label']]

,sentence,label
7207,it is put a picture of bill on your desk befor...,0
2840,carrie touched the stick against the cat .,0
4507,"mary sang a song , but lee did never .",0
440,mickey teamed with the women up .,0
346,john write books .,0


In [0]:
# get the list of sentences and their lables

sentences = df.sentence.values
labels = df.label.values

## Using BERT tokenizer
we need to use Bert tokenizer before feeding to BERT

In [10]:
from transformers import BertTokenizer

print('Loading Bert tokenizer.....')
# use the uncased pretrained version of the BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading Bert tokenizer.....


In [11]:
# Apply to one sentence and see
print('original sentence', sentences[0])
print('tokenized', tokenizer.tokenize(sentences[0]))

# sentence mapped to token ids
print('Token ids: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

original sentence our friends wo n't buy this analysis , let alone the next one we propose .
tokenized ['our', 'friends', 'wo', 'n', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
Token ids:  [2256, 2814, 24185, 1050, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]


The above can be done using `tokenizer.encode(sentences[0])`

In [12]:
print(sentences[0])
tokenizer.encode(sentences[0])

our friends wo n't buy this analysis , let alone the next one we propose .


[101,
 2256,
 2814,
 24185,
 1050,
 1005,
 1056,
 4965,
 2023,
 4106,
 1010,
 2292,
 2894,
 1996,
 2279,
 2028,
 2057,
 16599,
 1012,
 102]

## Tokenize Dataset

In [13]:
# maximum sentence length for padding and truncating

max_len = 0
for sent in sentences:
  # tokenize the text and add the '[CLS]' and '[SEP] tokens
  input_ids = tokenizer.encode(sent, add_special_tokens=True)

  # update the max_len
  max_len = max(max_len, len(input_ids))

print('Max sentence length', max_len)

Max sentence length 47


In case there are some bigger sentences in text, maximum lenght will be set as `64` instead of `47`

Now we're ready to perform the real tokenization.

The `tokenizer.encode_plus` function combines multiple steps for us:

1. Split the sentence into tokens.
2. Add the special `[CLS]` and `[SEP]` tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from `[PAD]` tokens.
  * Mask: 1, means real token
  * Mask: 0, mean padded token

6. It returns a dictionary of input_ids, token_type_ids and the attention_mask

In [14]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  our friends wo n't buy this analysis , let alone the next one we propose .
Token IDs: tensor([  101,  2256,  2814, 24185,  1050,  1005,  1056,  4965,  2023,  4106,
         1010,  2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [15]:
tokenizer.encode_plus(sentences[0])

{'input_ids': [101, 2256, 2814, 24185, 1050, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
input_ids

tensor([[ 101, 2256, 2814,  ...,    0,    0,    0],
        [ 101, 2028, 2062,  ...,    0,    0,    0],
        [ 101, 2028, 2062,  ...,    0,    0,    0],
        ...,
        [ 101, 2009, 2003,  ...,    0,    0,    0],
        [ 101, 1045, 2018,  ...,    0,    0,    0],
        [ 101, 2054, 2035,  ...,    0,    0,    0]])

## Training and Validation splits

In [19]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a Tensor Dataset
dataset  = TensorDataset(input_ids, attention_masks, labels)

# create a 90-10 split for train-validation
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# divide the dataset by randomly selecting samples
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5} validation samples'.format(val_size))

7,695 training samples
  856 validation samples


create an iterator for a dataset using the torch DataLoader class. This helps to save on memory

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# batch size is requires. For fine tuning BERT, author recommends a batch size of 16 or 32
batch_size=32

# Data loader for training dataset
train_dataloader  = DataLoader(
                      train_dataset,
                      batch_size = batch_size,
                      sampler = RandomSampler(train_dataset)
                      )

# Data loader for validation
# the order does not matter and hence we will just read sequentially
validation_dataloader = DataLoader(
                          val_dataset,
                          batch_size = batch_size,
                          sampler = SequentialSampler(val_dataset) # pulls batches sequentially
                            )

## Train Classification model

For this task, 
1. we first want to modify the `pre-trained BERT model` to give outputs for classification
2. and then we want to continue training the model on our dataset until that the entire model, end-to-end, is well-suited for our task.

Thankfully, the **huggingface** pytorch implementation includes a set of interfaces designed for a variety of NLP tasks. **Though these interfaces are all built on top of a trained BERT model, each has different top layers and output types designed to accomodate their specific NLP task**

Here is the current list of classes provided for fine-tuning:

* BertModel
* BertForPreTraining
* BertForMaskedLM
* BertForNextSentencePrediction
* **BertForSequenceClassification** - The one we'll use.
* BertForTokenClassification
* BertForQuestionAnswering

In [21]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# load the pretrained BERT model with a single linear classification at the top

model = BertForSequenceClassification.from_pretrained(
              'bert-base-uncased', # use the 12 layer BERT model with uncased vocab
              num_labels = 2, # for binary_classification
              output_attentions = False, # whether the model returns attention weights
              output_hidden_states = False, # whether the model outputs all hidden states
        )

# Tell pytorch to run this model on GPU
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Explore all the model parameters by name
Names and dimensions of the weights for:
1. The embedding layer
2. the first of the 12 transformers
3. Output layer

In [22]:
# get the model parameters as a list of tuples
params = list(model.named_parameters())

print('The BERT model has {:} parameters'.format(len(params)))

print('\n--------Embedding layer-------------\n')

for p in params[0:5]:
  #print('{} ------------------------ {}'.format(p[0], str(tuple(p[1].size()))))
  # Using {:<55}  {:>12} is text aligment
  # {:<55}: allocate 55 spaces to the left for the text, so this aligns from the left
  # {:>12}: allocate 12 spaces starting from right, so this aligns from the right
  print('{:<55}  {:>12}'.format(p[0], str(tuple(p[1].size()))))

print('\n----------First Transformer----------------\n')
for p in params[5:21]:
  #print('{} ------------------------- {}'.format(p[0], str(tuple(p[1].size()))))
  print('{:<55}  {:>12}'.format(p[0], str(tuple(p[1].size()))))

print('\n-----------------Output Layer--------------------\n')
for p in params[-4:]:
  #print('{} -------------------------{}'.format(p[0], str(tuple(p[1].size()))))
  print('{:<55}  {:>12}'.format(p[0], str(tuple(p[1].size()))))


The BERT model has 201 parameters

--------Embedding layer-------------

bert.embeddings.word_embeddings.weight                   (30522, 768)
bert.embeddings.position_embeddings.weight                 (512, 768)
bert.embeddings.token_type_embeddings.weight                 (2, 768)
bert.embeddings.LayerNorm.weight                               (768,)
bert.embeddings.LayerNorm.bias                                 (768,)

----------First Transformer----------------

bert.encoder.layer.0.attention.self.query.weight           (768, 768)
bert.encoder.layer.0.attention.self.query.bias                 (768,)
bert.encoder.layer.0.attention.self.key.weight             (768, 768)
bert.encoder.layer.0.attention.self.key.bias                   (768,)
bert.encoder.layer.0.attention.self.value.weight           (768, 768)
bert.encoder.layer.0.attention.self.value.bias                 (768,)
bert.encoder.layer.0.attention.output.dense.weight         (768, 768)
bert.encoder.layer.0.attention.output.den

## Optimizer and Learning Rate scheduler

Model is loaded. Now we need to grab the training `hyperparameters` from within the stored model

For the purpose of tuning, it is recommended to choose from the following values based on from Appendix A.3 of the [BERT paper]((https://arxiv.org/pdf/1810.04805.pdf):

>- **Batch Size**: 16, 32
- **Learning Rate(Adam)**: 5e-5, 3e-5, 2e-5
- **Number of epochs**: 2, 4

The epsilon parameter `eps = 1e-8` is "a very small number to prevent any division by zero in the implementation" (from [here](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/))

In [0]:
# AdamW is a class from the hugging face library
# Adam optimizer with weight decay fix
# the parameters passed to the optmizer are the model parameters

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # initial learning rate
                  eps = 1e-8 # default
)

Define the `scheduler` which handles the 'learning rate decay`.
When training a NN, you want to take large steps initially for the  initial learning decay rate and take progessively smaller steps as we horne in on the optimal value for the weights

In [0]:
from transformers import get_linear_schedule_with_warmup

# number of training epochs
epochs =4

# Total number of training steps = [number of batches]  X [number of epochs]
# this is not the same as the number of training samples

total_steps = len(train_dataloader) * epochs

# create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
                                          optimizer,
                                          num_warmup_steps = 0, # default value
                                          num_training_steps = total_steps
                                          )

## Training Loop

Below is our training loop. There's a lot going on, but fundamentally for each pass in our loop we have a trianing phase and a validation phase. 


**Training:**
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Clear out the gradients calculated in the previous pass. 
    - In pytorch the gradients accumulate by default (useful for things like RNNs) unless you explicitly clear them out.
- Forward pass (feed input data through the network)
- Backward pass (backpropagation)
- Tell the network to update parameters with optimizer.step()
- Track variables for monitoring progress

**Evalution:**
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Forward pass (feed input data through the network)
- Compute loss on our validation data and track variables for monitoring progress

define a helper function for accuracy

In [0]:
import numpy as np

# find accuracy of prediction vs labels

def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# helper function for elaped time

import time
import datetime

def format_time(elapsed):
  '''
  Takes time in seconds and returns
  a string in hh:mm:ss
  '''
  elapsed_rounded = int(round(elapsed))

  # format in hh:mm:ss
  return str(datetime.timedelta(seconds = elapsed_rounded))

## Kick of training

In [27]:
import numpy as np
import random

seed_val = 42

epochs=4

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings
training_stats= []

# measure total time take for the whole run
total_t0 = time.time()

# for each epoch
for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================

    # perform one full pass on the training set

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # measure how long the training epoch takes
    t0 = time.time()

    # reset the training loss for this epoch
    total_train_loss = 0

    # put the model in 'train' mode
    model.train()

    # for each batch of the training data
    for step, batch in enumerate(train_dataloader):
        
        # progress update every 40 batches
        if step % 40 == 0 and not step == 0:
          # calculate the elaped time
          elapsed = format_time(time.time() - t0)

          # report progress
          print('Batch: {} of {}.. Elaped time: {:}'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass

        model.zero_grad()
        
        # Perform a forward pass (evaluate the model on this training batch).
        loss, logits = model(
                            b_input_ids,
                            attention_mask = b_input_mask,
                            labels = b_labels,
                            token_type_ids = None
                          )
        
        # accumulate the training loss over all batches
        total_train_loss += loss.item()

        # perform backward pass to calculate the gradients
        loss.backward()

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.

        optimizer.step()

        # update learning rate        
        scheduler.step()

    # Calculate the average loss across all batches
    avg_train_loss = total_train_loss / len(train_dataloader)

    # measure how long this epoch took
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # put the model in evalaution mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # evaluate validation set for each epoch

    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.

            (loss, logits) = model(
                                  b_input_ids,
                                  attention_mask = b_attention_mask,
                                  labels = b_labels,
                                  token_type_ids = None
                                )
            
        # accumulate the validation loss
        total_eval_loss += loss.item()

        # move the logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids =  b_labels.to('cpu').numpy()

        # calaculate the  accuracy for this batch and accumulate over all batches
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # report the final accuracy for this validation run
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print('Average Accuracy for this validation run is {:.2f}'.format(avg_val_accuracy))

    # calculate the average loss across all batches
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # measure time taken
    validation_time = format_time(time.time() - t0)

    print('Average validation loss: {:.2f}'.format(avg_val_loss))
    print('Time taken for validation {}'.format(validation_time))

    training_stats.append(
                {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time   
               }
                )
    
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Batch: 40 of 241.. Elaped time: 0:00:24
Batch: 80 of 241.. Elaped time: 0:00:48
Batch: 120 of 241.. Elaped time: 0:01:13
Batch: 160 of 241.. Elaped time: 0:01:37
Batch: 200 of 241.. Elaped time: 0:02:02
Batch: 240 of 241.. Elaped time: 0:02:26

  Average training loss: 0.51
  Training epoch took: 0:02:26

Running Validation...
Average Accuracy for this validation run is 0.83
Average validation loss: 0.40
Time taken for validation 0:00:05

======== Epoch 2 / 2 ========
Training...
Batch: 40 of 241.. Elaped time: 0:00:24
Batch: 80 of 241.. Elaped time: 0:00:49
Batch: 120 of 241.. Elaped time: 0:01:13
Batch: 160 of 241.. Elaped time: 0:01:38
Batch: 200 of 241.. Elaped time: 0:02:02
Batch: 240 of 241.. Elaped time: 0:02:27

  Average training loss: 0.32
  Training epoch took: 0:02:27

Running Validation...
Average Accuracy for this validation run is 0.86
Average validation loss: 0.36
Time taken for validation 0:00:05

Training complete!
Total training took 0:05:05 (h:mm:ss)


## Summary of the training process

In [28]:
import pandas as pd
pd.set_option('precision', 2)

# create a dataframe from the training stats
df_stats =  pd.DataFrame(data = training_stats)

df_stats

,epoch,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
0,1,0.51,0.40,0.83,0:02:26,0:00:05
1,2,0.32,0.36,0.86,0:02:27,0:00:05
